# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [1]:
import gradio as gr
import openai
import tempfile

# Set your OpenAI API key
openai.api_key = "YOur APi HeRE"

def process_file_and_prompt(file, instructions):
    # Save the uploaded file to a temporary location
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(file.read())
        tmp_file_path = tmp_file.name

    # Read the content of the uploaded file
    with open(tmp_file_path, 'r') as f:
        file_content = f.read()

    # Combine file content with instructions
    prompt = f"{instructions}\n\nFile Content:\n{file_content}"

    # Call OpenAI's API to get a response
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response['choices'][0]['message']['content']

with gr.Blocks() as interface:
    gr.Markdown("## File Upload Assistant")
    
    with gr.Row():
        file_input = gr.File(label="Upload a file of your interest", type="file")
        instructions_input = gr.Textbox(label="Add Instructions to the prompt", lines=4)
    
    submit_button = gr.Button("Submit")
    output_text = gr.Textbox(label="Assistant Response", lines=10)

    submit_button.click(process_file_and_prompt, inputs=[file_input, instructions_input], outputs=output_text)

interface.launch(share=True)

/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


ValueError: Invalid value for parameter `type`: file. Please choose from one of: ['filepath', 'binary']

In [2]:
import gradio as gr
import openai
import tempfile

# Set your OpenAI API key
openai.api_key = "YOur APi HeRE"

def process_file_and_prompt(file, instructions):
    # Save the uploaded file to a temporary location
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(file.read())
        tmp_file_path = tmp_file.name

    # Read the content of the uploaded file
    with open(tmp_file_path, 'r') as f:
        file_content = f.read()

    # Combine file content with instructions
    prompt = f"{instructions}\n\nFile Content:\n{file_content}"

    # Call OpenAI's API to get a response
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response['choices'][0]['message']['content']

with gr.Blocks() as interface:
    gr.Markdown("## File Upload Assistant")
    
    with gr.Row():
        # Change type to "binary" or "filepath"
        file_input = gr.File(label="Upload a file of your interest", type="binary")
        instructions_input = gr.Textbox(label="Add Instructions to the prompt", lines=4)
    
    submit_button = gr.Button("Submit")
    output_text = gr.Textbox(label="Assistant Response", lines=10)

    submit_button.click(process_file_and_prompt, inputs=[file_input, instructions_input], outputs=output_text)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://f2ad3db730ee18ed85.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/blocks.py", line 1590, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/majed-abdulkare

In [3]:
import gradio as gr
import openai
import tempfile

# Set your OpenAI API key
openai.api_key = "YOur APi HeRE"

def process_file_and_prompt(file, instructions):
    # Save the uploaded file to a temporary location
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(file)  # Directly write the bytes to the temporary file
        tmp_file_path = tmp_file.name

    # Read the content of the uploaded file
    with open(tmp_file_path, 'r') as f:
        file_content = f.read()

    # Combine file content with instructions
    prompt = f"{instructions}\n\nFile Content:\n{file_content}"

    # Call OpenAI's API to get a response
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response['choices'][0]['message']['content']

with gr.Blocks() as interface:
    gr.Markdown("## File Upload Assistant")
    
    with gr.Row():
        file_input = gr.File(label="Upload a file of your interest", type="binary")
        instructions_input = gr.Textbox(label="Add Instructions to the prompt", lines=4)
    
    submit_button = gr.Button("Submit")
    output_text = gr.Textbox(label="Assistant Response", lines=10)

    submit_button.click(process_file_and_prompt, inputs=[file_input, instructions_input], outputs=output_text)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://b8c0809b63c5f52f10.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/blocks.py", line 1590, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/majed-abdulkare

In [4]:
import gradio as gr
import openai
import tempfile

# Set your OpenAI API key
openai.api_key = "YOur APi HeRE"

def process_file_and_prompt(file, instructions):
    # Save the uploaded file to a temporary location
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(file)  # Directly write the bytes to the temporary file
        tmp_file_path = tmp_file.name

    # Attempt to read the content of the uploaded file
    try:
        with open(tmp_file_path, 'r', encoding='utf-8') as f:
            file_content = f.read()
    except UnicodeDecodeError:
        # If UTF-8 fails, try a different encoding or handle the binary file
        try:
            with open(tmp_file_path, 'r', encoding='ISO-8859-1') as f:
                file_content = f.read()
        except Exception as e:
            return f"Error reading file: {e}"

    # Combine file content with instructions
    prompt = f"{instructions}\n\nFile Content:\n{file_content}"

    # Call OpenAI's API to get a response
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response['choices'][0]['message']['content']

with gr.Blocks() as interface:
    gr.Markdown("## File Upload Assistant")
    
    with gr.Row():
        file_input = gr.File(label="Upload a file of your interest", type="binary")
        instructions_input = gr.Textbox(label="Add Instructions to the prompt", lines=4)
    
    submit_button = gr.Button("Submit")
    output_text = gr.Textbox(label="Assistant Response", lines=10)

    submit_button.click(process_file_and_prompt, inputs=[file_input, instructions_input], outputs=output_text)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://34483b8b93f4b99612.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/blocks.py", line 1590, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/majed-abdulkare

In [5]:
import gradio as gr
import openai
import tempfile

openai.api_key = "YOur APi HeRE"

def process_file_and_prompt(file, instructions):
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(file) 
        tmp_file_path = tmp_file.name

    try:
        with open(tmp_file_path, 'r', encoding='utf-8') as f:
            file_content = f.read()
    except UnicodeDecodeError:
        try:
            with open(tmp_file_path, 'r', encoding='ISO-8859-1') as f:
                file_content = f.read()
        except Exception as e:
            return f"Error reading file: {e}"

    max_input_length = 2000  
    if len(file_content) > max_input_length:
        file_content = file_content[:max_input_length] 

    prompt = f"{instructions}\n\nFile Content:\n{file_content}"

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response['choices'][0]['message']['content']
    except openai.error.RateLimitError as e:
        return f"Rate limit exceeded: {e}"

with gr.Blocks() as interface:
    gr.Markdown("## File Upload Assistant")
    
    with gr.Row():
        file_input = gr.File(label="Upload a file of your interest", type="binary")
        instructions_input = gr.Textbox(label="Add Instructions to the prompt", lines=4)
    
    submit_button = gr.Button("Submit")
    output_text = gr.Textbox(label="Assistant Response", lines=10)

    submit_button.click(process_file_and_prompt, inputs=[file_input, instructions_input], outputs=output_text)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://3be975e4295f5ecab5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [6]:
import gradio as gr
import openai

openai.api_key = "YOur APi HeRE"

def chat_with_assistant(user_input):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": user_input}
        ]
    )
    return response['choices'][0]['message']['content']

with gr.Blocks() as interface:
    gr.Markdown("## Chat with Your Assistant")
    
    with gr.Row():
        user_input = gr.Textbox(label="Type your message here", lines=4)
        submit_button = gr.Button("Send")
    
    output_text = gr.Textbox(label="Assistant Response", lines=10)

    submit_button.click(chat_with_assistant, inputs=user_input, outputs=output_text)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://ea545112365c78cd1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [1]:
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Example usage:
forecast = get_weather_forecast(52.52, 13.41)
print(forecast)

{'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.06699562072753906, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2024-06-17T00:00', '2024-06-17T01:00', '2024-06-17T02:00', '2024-06-17T03:00', '2024-06-17T04:00', '2024-06-17T05:00', '2024-06-17T06:00', '2024-06-17T07:00', '2024-06-17T08:00', '2024-06-17T09:00', '2024-06-17T10:00', '2024-06-17T11:00', '2024-06-17T12:00', '2024-06-17T13:00', '2024-06-17T14:00', '2024-06-17T15:00', '2024-06-17T16:00', '2024-06-17T17:00', '2024-06-17T18:00', '2024-06-17T19:00', '2024-06-17T20:00', '2024-06-17T21:00', '2024-06-17T22:00', '2024-06-17T23:00', '2024-06-18T00:00', '2024-06-18T01:00', '2024-06-18T02:00', '2024-06-18T03:00', '2024-06-18T04:00', '2024-06-18T05:00', '2024-06-18T06:00', '2024-06-18T07:00', '2024-06-18T08:00', '2024-06-18T09:00', '2024-06-18T10:00', '2024-06-18T11:00', '2024-06-18T12:00'

In [7]:
import gradio as gr
import requests

def get_weather_forecast(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True  # Get current weather
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        weather_data = response.json()
        current_weather = weather_data['current_weather']
        return f"Temperature: {current_weather['temperature']}°C\n" \
               f"Wind Speed: {current_weather['windspeed']} km/h\n" \
               f"Wind Direction: {current_weather['winddirection']}°\n" \
               f"Weather Code: {current_weather['weathercode']}"
    else:
        return "Error fetching weather data. Please check the coordinates."

with gr.Blocks() as interface:
    gr.Markdown("## Weather Assistant")
    
    with gr.Row():
        latitude_input = gr.Textbox(label="Enter Latitude", placeholder="e.g., 52.52")
        longitude_input = gr.Textbox(label="Enter Longitude", placeholder="e.g., 13.41")
    
    submit_button = gr.Button("Get Weather")
    output_text = gr.Textbox(label="Weather Forecast", lines=10)

    submit_button.click(get_weather_forecast, inputs=[latitude_input, longitude_input], outputs=output_text)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7870
* Running on public URL: https://d4eaad23388f95259f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import gradio as gr
import requests
import openai

openai.api_key = "YOur APi HeRE"

def get_weather_forecast(location):
    geocode_url = f"https://geocoding-api.open-meteo.com/v1/search?name={location}"
    geocode_response = requests.get(geocode_url)
    
    if geocode_response.status_code == 200:
        geocode_data = geocode_response.json()
        if geocode_data['results']:
            latitude = geocode_data['results'][0]['latitude']
            longitude = geocode_data['results'][0]['longitude']
            
            weather_url = "https://api.open-meteo.com/v1/forecast"
            params = {
                "latitude": latitude,
                "longitude": longitude,
                "current_weather": True
            }
            weather_response = requests.get(weather_url, params=params)
            
            if weather_response.status_code == 200:
                weather_data = weather_response.json()
                current_weather = weather_data['current_weather']
                return f"Current weather in {location}:\n" \
                       f"Temperature: {current_weather['temperature']}°C\n" \
                       f"Wind Speed: {current_weather['windspeed']} km/h\n" \
                       f"Wind Direction: {current_weather['winddirection']}°\n" \
                       f"Weather Code: {current_weather['weathercode']}"
            else:
                return "Error fetching weather data."
        else:
            return "Location not found."
    else:
        return "Error fetching location data."

def chat_with_assistant(user_input):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": user_input}
        ]
    )
    assistant_response = response['choices'][0]['message']['content']
    
    if "weather" in user_input.lower():
        location = user_input.split("weather in")[-1].strip()  
        weather_info = get_weather_forecast(location)
        return f"{assistant_response}\n\n{weather_info}"
    
    return assistant_response

with gr.Blocks() as interface:
    gr.Markdown("## Chat with Your Weather Assistant")
    
    with gr.Row():
        user_input = gr.Textbox(label="Type your message here", lines=4)
        submit_button = gr.Button("Send")
    
    output_text = gr.Textbox(label="Assistant Response", lines=10)

    submit_button.click(chat_with_assistant, inputs=user_input, outputs=output_text)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7871
* Running on public URL: https://afadf536cde64bca8d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/blocks.py", line 2043, in process_api
    result = await self.call_function(
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/gradio/blocks.py", line 1590, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/Users/majed-abdulkareem/opt/anaconda3/envs/FTF2/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/Users/majed-abdulkare

### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality